In [6]:
# import libraries
import numpy as np
import pandas as pd
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [7]:
fd = pd.read_csv('final_data_encoded.csv', header=0)
fd = fd.drop('Unnamed: 0', 1)
fd = fd.drop('Unnamed: 0.1', 1)
fd.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,Motor Vehicle Theft,Larceny-Theft,Arson,Law Enforcement,Police Dept,Sheriff's Dept,Prosecution,Public Defense,Probation Dept,address
0,11016594,0.0276,0,1.0,2.0,3.0,4.0,2.0,1684.0,1684.0,...,1,1,1,1,0,1,0,0,2,"16358 Halsey St, Granada Hills CA 91344-3030"
1,12098116,-0.0040,0,1.0,3.0,2.0,4.0,3.0,2217.0,2217.0,...,1,1,1,1,0,1,0,0,2,"1211 Nithsdale Rd, Pasadena CA 91105-1463"
2,12643413,0.0218,1,1.0,2.0,2.0,4.0,2.0,839.0,839.0,...,1,1,1,1,0,1,0,0,2,"28974 S Western Ave, Rancho Palos Verdes CA 90..."
3,11509835,-0.2705,1,1.0,4.0,4.0,1.0,4.0,3067.0,3067.0,...,1,1,1,1,0,1,0,0,2,"2288 Monterey Blvd, Hermosa Beach CA 90254-2632"
4,12286022,0.0440,1,1.0,1.0,2.0,7.0,1.0,1297.0,1297.0,...,1,1,1,1,0,1,0,0,2,"502 N Mayo Ave, Compton CA 90221-2741"


In [12]:
X = fd.drop(['address','logerror'], 1)
y = fd['logerror']

In [11]:
X.head(5)

,parcelid,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,Burglary,Motor Vehicle Theft,Larceny-Theft,Arson,Law Enforcement,Police Dept,Sheriff's Dept,Prosecution,Public Defense,Probation Dept
0,11016594,0,1.0,2.0,3.0,4.0,2.0,1684.0,1684.0,6037.0,...,2,1,1,1,1,0,1,0,0,2
1,12098116,0,1.0,3.0,2.0,4.0,3.0,2217.0,2217.0,6037.0,...,2,1,1,1,1,0,1,0,0,2
2,12643413,1,1.0,2.0,2.0,4.0,2.0,839.0,839.0,6037.0,...,2,1,1,1,1,0,1,0,0,2
3,11509835,1,1.0,4.0,4.0,1.0,4.0,3067.0,3067.0,6037.0,...,2,1,1,1,1,0,1,0,0,2
4,12286022,1,1.0,1.0,2.0,7.0,1.0,1297.0,1297.0,6037.0,...,2,1,1,1,1,0,1,0,0,2


In [13]:
y

0        0.0276
1       -0.0040
2        0.0218
3       -0.2705
4        0.0440
          ...  
90270   -0.0070
90271    0.0178
90272    0.0030
90273   -0.0111
90274    0.0010
Name: logerror, Length: 90275, dtype: float64

In [62]:
0.06850220739022277 / ((y.max()) + abs(y.min()))

0.007332713272342407

In [25]:
# Dummy classifier 
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

# split 
X_train, X_test, y_train, y_test = train_test_split (
    X, y, test_size=0.25, random_state=42
)

# build
dummy_regressor = DummyRegressor(strategy="mean")

# train
dummy_regressor.fit(X_train, y_train)



DummyRegressor()

In [26]:
# score - mean absolute error (MAE) 
from sklearn import metrics 

y_pred = dummy_regressor.predict(X_test) 

mae = metrics.mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.06883763653277597


In [35]:
# linear regression 
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

# MinMaxScaler 
scaler = MinMaxScaler(feature_range=(0,1))
X_train_scaled = scaler.fit_transform(X_train)

# train
reg = LinearRegression().fit(X_train_scaled, y_train)

X_test_scaled = scaler.transform(X_test)
y_pred = reg.predict(X_test_scaled)

# score
mae = metrics.mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.068718758944467


In [50]:
# xgboost
import xgboost as xgb

# # convert data set into optimized data structure Dmatrix 
# data_matrix = xgb.DMatrix(data=X, label=y)

# build
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                         max_depth=6, alpha=10, n_estimators=100)

# train
xg_reg.fit(X_train_scaled, y_train)

# score 
y_pred = xg_reg.predict(X_test_scaled)
mae = metrics.mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.06850220739022277


In [ ]:
#Assume logerror is ranging from -4.6 to 4.7 and the error is 0.069, it means on average your 
#prediction is 0.069 different from ground truth.

#For example, there is a house, the ground truth is 4.5 logerror for it, the model can 
#predict 4.44 or 4.56. Of course, this is the average performance, the model can be very good on 
#this data point, e.g., producing perfect prediction or be very bad at this data point, make much 
#more error than 0.06.

### logerror = Log(Zestimate) - Log(Sales price)
(0.068718758944467) = log(x) - log(3050000)
x = 3266960

3266960 - 3050000
x = 216960

In [ ]:
#8611 Bluffdale Dr, Sun Valley, CA 91352
x = 1071130
x = 71130
0.07113